# Qwen3

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

d:\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import torch

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print("Total VRAM:", torch.cuda.get_device_properties(0).total_memory / 1024**3, "GB")
    print("Allocated VRAM:", torch.cuda.memory_allocated(0) / 1024**3, "GB")
    print("Cached VRAM:", torch.cuda.memory_reserved(0) / 1024**3, "GB")
else:
    print("No GPU found.")

NVIDIA GeForce GTX 1650
Total VRAM: 3.99969482421875 GB
Allocated VRAM: 0.0 GB
Cached VRAM: 0.0 GB


## Getting Started

In [ ]:
model_name = "Qwen/Qwen3-1.7B"
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto" # run on GPU if available
)


d:\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
# prepare the model input
prompt = '''You are an expert in machine translation.
Your task is to translate English to Vietnamese and Japanese.
Here is your sentence: 
If your model is still being downloaded to the old location instead of your new HF_HOME path, it is likely because the environment variable is not set early enough or not set for the current Python process.'''
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

print(model_inputs)


{'input_ids': tensor([[151644,    872,    198,   2610,    525,    458,   6203,    304,   5662,
          14468,    624,   7771,   3383,    374,    311,  14683,   6364,    311,
          48477,    323,  10769,    624,   8420,    374,    697,  11652,     25,
            715,   2679,    697,   1614,    374,   2058,   1660,  23293,    311,
            279,   2310,   3728,   4518,    315,    697,    501,  50558,  28466,
           1815,     11,    432,    374,   4363,   1576,    279,   4573,   3890,
            374,    537,    738,   4124,   3322,    476,    537,    738,    369,
            279,   1482,  13027,   1882,     13, 151645,    198, 151644,  77091,
            198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}


In [4]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)


In [5]:
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

print(len(output_ids))

693


In [6]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:\n", thinking_content)
print("content:\n", content)

thinking content:
 <think>
Okay, let's tackle this translation. The user wants the English sentence translated into both Vietnamese and Japanese. First, I need to understand the original sentence thoroughly.

The sentence is about an issue with a model being downloaded to an old location instead of the new HF_HOME path. The problem is likely due to the environment variable not being set early enough or not set for the current Python process.

Starting with the Vietnamese translation. The key terms here are "model is still being downloaded," "old location," "new HF_HOME path," "environment variable," "not set early enough," and "current Python process."

I need to make sure technical terms like "HF_HOME" are kept as is, since it's a specific variable. The phrase "environment variable" in Vietnamese is "biến môi trường" or "biến môi trường hệ thống." "Not set early enough" could be "không được thiết lập sớm hơn cần thiết" or "không được thiết lập đủ sớm." 

For the structure, the sentenc

## Chatbot with history

In [3]:
class QwenChatbot: 
    def __init__(self, model_name = 'Qwen/Qwen3-0.6B'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.history = []
    def generate_response(self, question, thinking = True,):
        message = self.history + [{'role': 'user', 'content': question}]
        text = self.tokenizer.apply_chat_template(
            message,
            tokenize = False,
            add_generation_prompt = True,
            enable_thinking = thinking
        )
        ques = self.tokenizer(text, return_tensors = 'pt')
        response = self.model.generate(**ques, max_new_tokens = 32768)
        response_ids = response[0][len(ques.input_ids[0]):].tolist()
        ans = self.tokenizer.decode(response_ids, skip_special_tokens = True)

        self.history.append({'role':'user', 'content': ques})
        self.history.append({'role': 'assistant', 'content': ans})

        return ans


In [4]:
chatbot = QwenChatbot(model_name = "Qwen/Qwen3-1.7B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
prompt = "You are an expert in machine translation, your task is to translate from english to vietnamese and japanese, reply U if understand your task"
print(f"User: {prompt}")
r0 = chatbot.generate_response(prompt, thinking=False)
print(f"Bot: {r0}")
print("-----------------------------------------")


q1 = "I love cats and dogs"
print(f"User: {q1}") 
r1 = chatbot.generate_response(q1, thinking = False)
print(f"Bot: {r1}")
print("------------------------------------------")

User: You are an expert in machine translation, your task is to translate from english to vietnamese and japanese, reply U if understand your task
Bot: U
-----------------------------------------
User: I love cats and dogs


KeyboardInterrupt: 

In [ ]:
q2 ='''Certainly!
The line `if __name__ == "__main__":` is a special Python check.
- When you run the file directly (for example: python getting_started.ipynb or run all cells in Jupyter), Python sets the variable __name__ to "__main__".
- When you import this file into another Python file (for example: import getting_started), __name__ is set to the file's name, not "__main__".
So:
- The code inside this block will only run if you execute the file directly.
- If you import the file as a module, the code inside this block will not run automatically.'''
print(f"User: {q2}") 
r2 = chatbot.generate_response(q2, thinking = False)
print(f"Bot: {r2}")
print("------------------------------------------")

User: Certainly!
The line `if __name__ == "__main__":` is a special Python check.
- When you run the file directly (for example: python getting_started.ipynb or run all cells in Jupyter), Python sets the variable __name__ to "__main__".
- When you import this file into another Python file (for example: import getting_started), __name__ is set to the file's name, not "__main__".
So:
- The code inside this block will only run if you execute the file directly.
- If you import the file as a module, the code inside this block will not run automatically.
Bot: Đúng! `if __name__ == "__main__":` là một kiểm tra đặc biệt trong Python.  
- Khi bạn chạy file trực tiếp (ví dụ: `python getting_started.ipynb` hoặc chạy tất cả các dòng trong Jupyter), Python sẽ đặt `__name__` vào `"__main__"`.
- Khi bạn thêm file này vào một file Python khác (ví dụ: `import getting_started`), `__name__` sẽ được đặt vào tên của file đó, thay vì `"__main__"`.
- Do đó:
  - Code trong đoạn này sẽ chỉ chạy nếu bạn chạy fi

__Note:__

 _ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`_ 

The accelerate library by Hugging Face is designed to simplify and optimize running PyTorch (and some TensorFlow) models on different hardware (CPU, GPU, multi-GPU, TPU, etc.). Its main roles are:
- Automatic device placement: Easily move models and tensors to the right device (CPU, GPU, etc.) without manual code changes.
- Distributed training: Run your code on multiple GPUs or even multiple machines with minimal changes.
- Mixed precision: Enable faster training/inference with less memory by using mixed precision (float16/float32).
- Seamless integration: Works with Hugging Face Transformers and Diffusers, and can be used with your own PyTorch code.
In your case, using device_map="auto" in from_pretrained requires accelerate to handle device placement automatically.

-----

_What are GPU 0 and GPU 1 means?_

Gpu 0 is the integrated graphics that are on your cpu. This gives basic functionality. You don't need this if you have a dedicated card.

Gpu 1 is your dedicated card. Monitors should be plugged into this. This gives gaming performance (card model depending)

--------
_"Some parameters are on the meta device because they were offloaded to the cpu." Thế này nghĩa là gì?_ 

Thông báo "Some parameters are on the meta device because they were offloaded to the cpu." nghĩa là:

- Một số tham số (parameters) của mô hình không được nạp trực tiếp vào GPU hoặc RAM, mà đang ở trạng thái "meta".
- "meta device" là một thiết bị ảo trong PyTorch, dùng để đại diện cho các tham số chưa thực sự được lưu trữ trên bất kỳ thiết bị vật lý nào (CPU hoặc GPU).
- Điều này thường xảy ra khi bạn dùng `device_map="auto"` hoặc các chế độ offload để tiết kiệm bộ nhớ, đặc biệt với các mô hình lớn.
- Khi cần thiết, các tham số này sẽ được "offload" (chuyển) sang CPU để sử dụng, thay vì giữ toàn bộ mô hình trên GPU (do giới hạn VRAM).

Tóm lại: \
Một phần mô hình của bạn đang không nằm trên GPU mà được lưu tạm trên CPU hoặc ở trạng thái "meta" để tiết kiệm tài nguyên. Khi cần tính toán, chúng sẽ được chuyển về CPU hoặc GPU. Đây là cơ chế tối ưu bộ nhớ của Hugging Face Transformers khi làm việc với mô hình lớn.